In [2]:
from zipfile import ZipFile

file_name = 'khateeb_data'

with ZipFile(file_name, 'r') as f:
  f.extractall()
  print("Done")

FileNotFoundError: [Errno 2] No such file or directory: 'khateeb_data'

In [2]:
import os

current_directory = os.getcwd()
path = os.path.join(current_directory, "khateeb_data")
print(path)

C:\Users\Moath\Downloads\khateeb_data


In [3]:
pip install tensorflow 


  Using cached tensorflow-2.12.0-cp310-cp310-win_amd64.whl (1.9 kB)
     ------------------------------------ 272.8/272.8 MB 646.6 kB/s eta 0:00:00
     ---------------------------------------- 4.1/4.1 MB 4.6 MB/s eta 0:00:00
     -------------------------------------- 65.5/65.5 kB 708.3 kB/s eta 0:00:00
  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
     ---------------------------------------- 1.5/1.5 MB 2.8 MB/s eta 0:00:00
     ---------------------------------------- 24.4/24.4 MB 3.8 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB 1.0 MB/s eta 0:00:00
     -------------------------------------- 126.5/126.5 kB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 2.0 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (p

In [5]:
pip install opencv-python

  Using cached opencv_python-4.7.0.72-cp37-abi3-win_amd64.whl (38.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import sklearn.metrics as metrics
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.applications import VGG16
import cv2
import imghdr
import os
import numpy as np

In [ ]:
rm train/.DS_Store

In [ ]:
rm test/.DS_Store

In [9]:
folders = ['train', 'test']
image_exts = ['jpeg','jpg', 'bmp', 'png']

for image_class in os.listdir(path): 
  print(image_class)
  for image in os.listdir(os.path.join(path, image_class)):
      image_path = os.path.join(path, image_class, image)
      try: 
          img = cv2.imread(image_path)
          tip = imghdr.what(image_path)
          if tip not in image_exts: 
              print('Image not in ext list {}'.format(image_path))
              os.remove(image_path)
      except Exception as e: 
          print('Issue with image {}'.format(image_path))
          # os.remove(image_path)

black_bishop
black_king
black_knight
black_pawn
black_queen
black_rook
empty
white_bishop
white_king
white_knight
white_pawn
white_queen
white_rook


In [27]:
trdata = ImageDataGenerator(rotation_range = 60, validation_split=0.2)
train_data_gen = trdata.flow_from_directory(directory="khateeb_data", target_size=(299,299), shuffle=True, class_mode="categorical", subset="training", batch_size = 20)
validation_data_gen = trdata.flow_from_directory(directory="khateeb_data", target_size=(299,299), shuffle=True, class_mode="categorical", subset="validation", batch_size = 20)

Found 1798 images belonging to 13 classes.
Found 437 images belonging to 13 classes.


In [16]:
from tensorflow.keras.applications import VGG16

vgg = VGG16(input_shape = (224,224,3), weights = "imagenet", include_top = False)
for layer in vgg.layers:
 layer.trainable = False

x = tf.keras.layers.Flatten()(vgg.output)
x = tf.keras.layers.Dense(720, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(720, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(720, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(720, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.Dense(13, activation = "softmax")(x)

model = Model(inputs = vgg.input, outputs = x)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [17]:
training_steps_per_epoch = np.ceil(train_data_gen.samples / 32)
validation_steps_per_epoch = np.ceil(validation_data_gen.samples / 32)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True,verbose=1)

model.fit(train_data_gen, steps_per_epoch = training_steps_per_epoch, validation_data=validation_data_gen, validation_steps=validation_steps_per_epoch, epochs=15, verbose=1, callbacks=[early_stopping_cb])

Epoch 1/15
57/57 [==============================] - 702s 12s/step - loss: 1.3737 - accuracy: 0.5695 - val_loss: 6.2816 - val_accuracy: 0.2929
Epoch 2/15
57/57 [==============================] - 640s 11s/step - loss: 0.5393 - accuracy: 0.8337 - val_loss: 2.8879 - val_accuracy: 0.5286
Epoch 3/15
57/57 [==============================] - 641s 11s/step - loss: 0.3627 - accuracy: 0.8882 - val_loss: 2.0862 - val_accuracy: 0.6110
Epoch 4/15
57/57 [==============================] - 639s 11s/step - loss: 0.3474 - accuracy: 0.8971 - val_loss: 2.6472 - val_accuracy: 0.5515
Epoch 5/15
57/57 [==============================] - 638s 11s/step - loss: 0.2358 - accuracy: 0.9288 - val_loss: 2.0215 - val_accuracy: 0.6384
Epoch 6/15
57/57 [==============================] - 643s 11s/step - loss: 0.2834 - accuracy: 0.9227 - val_loss: 1.8223 - val_accuracy: 0.6316
Epoch 7/15
57/57 [==============================] - 640s 11s/step - loss: 0.2373 - accuracy: 0.9377 - val_loss: 1.6484 - val_accuracy: 0.6957
Epoch 

KeyboardInterrupt: 

In [21]:
from tensorflow.keras.applications import VGG19

vgg2 = VGG19(input_shape = (224,224,3), weights = "imagenet", include_top = False)
for layer in vgg.layers:
 layer.trainable = False

y = tf.keras.layers.Flatten()(vgg2.output)
y = tf.keras.layers.Dense(128, activation=tf.nn.swish, kernel_initializer="he_normal")(y)
y = tf.keras.layers.BatchNormalization()(y)
y = tf.keras.layers.Dropout(0.2)(y)
y = tf.keras.layers.Dense(128, activation=tf.nn.swish, kernel_initializer="he_normal")(y)
y = tf.keras.layers.BatchNormalization()(y)
y = tf.keras.layers.Dropout(0.2)(y)
y = tf.keras.layers.Dense(128, activation=tf.nn.swish, kernel_initializer="he_normal")(y)
y = tf.keras.layers.BatchNormalization()(y)
y = tf.keras.layers.Dropout(0.2)(y)
y = tf.keras.layers.Dense(64, activation=tf.nn.swish, kernel_initializer="he_normal")(y)
y = tf.keras.layers.Dense(13, activation = "softmax")(y)

model2 = Model(inputs = vgg2.input, outputs = y)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.95, nesterov=True)

model2.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

80134624/80134624 [==============================] - 8s 0us/step


In [22]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True,verbose=1)

model2.fit(train_data_gen, validation_data=validation_data_gen, epochs=12, verbose=1, callbacks=[early_stopping_cb])

Epoch 1/12
57/57 [==============================] - 1783s 31s/step - loss: 2.3825 - accuracy: 0.2108 - val_loss: 41.1772 - val_accuracy: 0.1510
Epoch 2/12
57/57 [==============================] - 1786s 31s/step - loss: 1.8407 - accuracy: 0.3204 - val_loss: 21.0104 - val_accuracy: 0.0801
Epoch 3/12
57/57 [==============================] - 1734s 30s/step - loss: 1.7000 - accuracy: 0.3587 - val_loss: 5.1337 - val_accuracy: 0.1556
Epoch 4/12
57/57 [==============================] - 1749s 31s/step - loss: 1.6176 - accuracy: 0.3771 - val_loss: 2.1906 - val_accuracy: 0.2449
Epoch 5/12
57/57 [==============================] - 1751s 31s/step - loss: 1.5541 - accuracy: 0.3960 - val_loss: 4.8963 - val_accuracy: 0.1739
Epoch 6/12
57/57 [==============================] - 1734s 31s/step - loss: 1.5627 - accuracy: 0.4027 - val_loss: 2.0432 - val_accuracy: 0.1968
Epoch 7/12
 1/57 [..............................] - ETA: 29:58 - loss: 1.4597 - accuracy: 0.3750

KeyboardInterrupt: 

In [4]:
trdata = ImageDataGenerator(rotation_range = 60, validation_split=0.2)
train_data_gen = trdata.flow_from_directory(directory="khateeb_data", target_size=(299,299), shuffle=True, class_mode="categorical", subset="training", batch_size = 20)
validation_data_gen = trdata.flow_from_directory(directory="khateeb_data", target_size=(299,299), shuffle=True, class_mode="categorical", subset="validation", batch_size = 20)

Found 1798 images belonging to 13 classes.
Found 437 images belonging to 13 classes.


In [6]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model

incep = InceptionV3(input_shape = (299,299,3), weights = "imagenet", include_top = False)
for layer in incep.layers:
 layer.trainable = False

y = tf.keras.layers.Flatten()(incep.output)
y = tf.keras.layers.Dense(32, activation=tf.nn.swish, kernel_initializer="he_normal")(y)
y = tf.keras.layers.BatchNormalization()(y)
y = tf.keras.layers.Dropout(0.2)(y)
y = tf.keras.layers.Dense(32, activation=tf.nn.swish, kernel_initializer="he_normal")(y)
y = tf.keras.layers.BatchNormalization()(y)
y = tf.keras.layers.Dropout(0.2)(y)
y = tf.keras.layers.Dense(32, activation=tf.nn.swish, kernel_initializer="he_normal")(y)
y = tf.keras.layers.BatchNormalization()(y)
y = tf.keras.layers.Dropout(0.2)(y)
y = tf.keras.layers.Dense(32, activation=tf.nn.swish, kernel_initializer="he_normal")(y)
y = tf.keras.layers.Dense(13, activation = "softmax")(y)

incep_model = Model(inputs = incep.input, outputs = y)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

incep_model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [8]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True,verbose=1)
incep_model.fit(train_data_gen, validation_data=validation_data_gen, epochs=12, verbose=1, callbacks=[early_stopping_cb])

Epoch 1/12
90/90 [==============================] - 536s 6s/step - loss: 2.3823 - accuracy: 0.1802 - val_loss: 2.8023 - val_accuracy: 0.1190
Epoch 2/12
90/90 [==============================] - 504s 6s/step - loss: 2.0285 - accuracy: 0.2280 - val_loss: 1.8740 - val_accuracy: 0.2517
Epoch 3/12
90/90 [==============================] - 503s 6s/step - loss: 1.8925 - accuracy: 0.2775 - val_loss: 1.9320 - val_accuracy: 0.1739
Epoch 4/12
90/90 [==============================] - 503s 6s/step - loss: 1.8383 - accuracy: 0.2987 - val_loss: 1.9894 - val_accuracy: 0.2288
Epoch 5/12
90/90 [==============================] - 502s 6s/step - loss: 1.7827 - accuracy: 0.3343 - val_loss: 1.8765 - val_accuracy: 0.2609
Epoch 6/12
90/90 [==============================] - 501s 6s/step - loss: 1.6833 - accuracy: 0.3459 - val_loss: 2.0687 - val_accuracy: 0.1808
Epoch 7/12
90/90 [==============================] - 503s 6s/step - loss: 1.6264 - accuracy: 0.3687 - val_loss: 2.0018 - val_accuracy: 0.2380
Epoch 7: earl

In [9]:
trdata = ImageDataGenerator(rotation_range = 60, validation_split=0.2)
train_data_gen = trdata.flow_from_directory(directory="khateeb_data", target_size=(224,224), shuffle=True, class_mode="categorical", subset="training", batch_size = 10)
validation_data_gen = trdata.flow_from_directory(directory="khateeb_data", target_size=(224,224), shuffle=True, class_mode="categorical", subset="validation", batch_size = 10)

Found 1798 images belonging to 13 classes.
Found 437 images belonging to 13 classes.


In [11]:
from tensorflow.keras.applications import VGG16

vgg = VGG16(input_shape = (224,224,3), weights = "imagenet", include_top = False)
for layer in vgg.layers:
 layer.trainable = False

x = tf.keras.layers.Flatten()(vgg.output)
x = tf.keras.layers.Dense(720, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(720, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(720, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(720, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.Dense(13, activation = "softmax")(x)

model = Model(inputs = vgg.input, outputs = x)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [12]:
training_steps_per_epoch = np.ceil(train_data_gen.samples / 32)
validation_steps_per_epoch = np.ceil(validation_data_gen.samples / 32)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True,verbose=1)

model.fit(train_data_gen, steps_per_epoch = training_steps_per_epoch, validation_data=validation_data_gen, validation_steps=validation_steps_per_epoch, epochs=15, verbose=1, callbacks=[early_stopping_cb])

Epoch 1/15
57/57 [==============================] - 198s 3s/step - loss: 3.4588 - accuracy: 0.2940 - val_loss: 23.1128 - val_accuracy: 0.0714
Epoch 2/15
57/57 [==============================] - 197s 3s/step - loss: 3.2994 - accuracy: 0.4261 - val_loss: 9.7720 - val_accuracy: 0.3071
Epoch 3/15
57/57 [==============================] - 196s 3s/step - loss: 2.8773 - accuracy: 0.4754 - val_loss: 5.4335 - val_accuracy: 0.3643
Epoch 4/15
57/57 [==============================] - 194s 3s/step - loss: 2.8386 - accuracy: 0.5423 - val_loss: 4.2415 - val_accuracy: 0.4357
Epoch 5/15
57/57 [==============================] - 196s 3s/step - loss: 2.3360 - accuracy: 0.5737 - val_loss: 2.0125 - val_accuracy: 0.4929
Epoch 6/15
57/57 [==============================] - 194s 3s/step - loss: 2.0262 - accuracy: 0.5739 - val_loss: 3.2312 - val_accuracy: 0.4214
Epoch 7/15
57/57 [==============================] - 194s 3s/step - loss: 1.6701 - accuracy: 0.6461 - val_loss: 1.8819 - val_accuracy: 0.5214
Epoch 8/15
5

In [13]:
trdata = ImageDataGenerator(rotation_range = 60, validation_split=0.2)
train_data_gen = trdata.flow_from_directory(directory="khateeb_data", target_size=(224,224), shuffle=True, class_mode="categorical", subset="training")
validation_data_gen = trdata.flow_from_directory(directory="khateeb_data", target_size=(224,224), shuffle=True, class_mode="categorical", subset="validation")

Found 1798 images belonging to 13 classes.
Found 437 images belonging to 13 classes.


In [14]:
from tensorflow.keras.applications import VGG16

vgg = VGG16(input_shape = (224,224,3), weights = "imagenet", include_top = False)
for layer in vgg.layers:
 layer.trainable = False

x = tf.keras.layers.Flatten()(vgg.output)
x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(13, activation = "softmax")(x)

model = Model(inputs = vgg.input, outputs = x)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [15]:
training_steps_per_epoch = np.ceil(train_data_gen.samples / 32)
validation_steps_per_epoch = np.ceil(validation_data_gen.samples / 32)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True,verbose=1)

model.fit(train_data_gen, steps_per_epoch = training_steps_per_epoch, validation_data=validation_data_gen, validation_steps=validation_steps_per_epoch, epochs=15, verbose=1, callbacks=[early_stopping_cb])

Epoch 1/15
57/57 [==============================] - 589s 10s/step - loss: 1.2346 - accuracy: 0.6702 - val_loss: 2.2747 - val_accuracy: 0.4073
Epoch 2/15
57/57 [==============================] - 582s 10s/step - loss: 0.3690 - accuracy: 0.9277 - val_loss: 1.3253 - val_accuracy: 0.5812
Epoch 3/15
57/57 [==============================] - 583s 10s/step - loss: 0.1736 - accuracy: 0.9655 - val_loss: 1.1863 - val_accuracy: 0.6018
Epoch 4/15
57/57 [==============================] - 582s 10s/step - loss: 0.0965 - accuracy: 0.9878 - val_loss: 1.0383 - val_accuracy: 0.6384
Epoch 5/15
57/57 [==============================] - 585s 10s/step - loss: 0.0750 - accuracy: 0.9855 - val_loss: 1.0207 - val_accuracy: 0.6842
Epoch 6/15
57/57 [==============================] - 583s 10s/step - loss: 0.0499 - accuracy: 0.9900 - val_loss: 1.0689 - val_accuracy: 0.6407
Epoch 7/15
57/57 [==============================] - 582s 10s/step - loss: 0.0314 - accuracy: 0.9972 - val_loss: 0.9844 - val_accuracy: 0.6796
Epoch 

In [16]:
from tensorflow.keras.applications import VGG16

vgg = VGG16(input_shape = (224,224,3), weights = "imagenet", include_top = False)
for layer in vgg.layers:
 layer.trainable = False

x = tf.keras.layers.Flatten()(vgg.output)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(13, activation = "softmax")(x)

model = Model(inputs = vgg.input, outputs = x)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [17]:

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True,verbose=1)

model.fit(train_data_gen, validation_data=validation_data_gen, epochs=15, verbose=1, callbacks=[early_stopping_cb])

Epoch 1/15
57/57 [==============================] - 616s 11s/step - loss: 0.9948 - accuracy: 0.7086 - val_loss: 1.7415 - val_accuracy: 0.5400
Epoch 2/15
57/57 [==============================] - 622s 11s/step - loss: 0.3216 - accuracy: 0.9321 - val_loss: 1.1987 - val_accuracy: 0.6293
Epoch 3/15
57/57 [==============================] - 667s 12s/step - loss: 0.1724 - accuracy: 0.9722 - val_loss: 1.0209 - val_accuracy: 0.6705
Epoch 4/15
57/57 [==============================] - 671s 12s/step - loss: 0.1131 - accuracy: 0.9816 - val_loss: 0.9940 - val_accuracy: 0.6957
Epoch 5/15
57/57 [==============================] - 645s 11s/step - loss: 0.0867 - accuracy: 0.9822 - val_loss: 1.0465 - val_accuracy: 0.6590
Epoch 6/15
57/57 [==============================] - 655s 12s/step - loss: 0.0662 - accuracy: 0.9889 - val_loss: 0.9594 - val_accuracy: 0.6819
Epoch 7/15
57/57 [==============================] - 649s 11s/step - loss: 0.0642 - accuracy: 0.9872 - val_loss: 0.9679 - val_accuracy: 0.7094
Epoch 

In [18]:
from tensorflow.keras.applications import VGG16

vgg = VGG16(input_shape = (224,224,3), weights = "imagenet", include_top = False)
for layer in vgg.layers:
 layer.trainable = False

x = tf.keras.layers.Flatten()(vgg.output)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(13, activation = "softmax")(x)

model = Model(inputs = vgg.input, outputs = x)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [19]:

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True,verbose=1)

model.fit(train_data_gen, validation_data=validation_data_gen, epochs=10, verbose=1, callbacks=[early_stopping_cb])

Epoch 1/10
57/57 [==============================] - 595s 10s/step - loss: 2.3496 - accuracy: 0.2108 - val_loss: 2.6816 - val_accuracy: 0.1556
Epoch 2/10
57/57 [==============================] - 579s 10s/step - loss: 1.8914 - accuracy: 0.3504 - val_loss: 1.7730 - val_accuracy: 0.3776
Epoch 3/10
57/57 [==============================] - 577s 10s/step - loss: 1.5851 - accuracy: 0.4333 - val_loss: 1.5232 - val_accuracy: 0.4439
Epoch 4/10
57/57 [==============================] - 578s 10s/step - loss: 1.4151 - accuracy: 0.4705 - val_loss: 1.4625 - val_accuracy: 0.4645
Epoch 5/10
57/57 [==============================] - 580s 10s/step - loss: 1.2438 - accuracy: 0.5317 - val_loss: 1.2718 - val_accuracy: 0.5195
Epoch 6/10
57/57 [==============================] - 577s 10s/step - loss: 1.1960 - accuracy: 0.5462 - val_loss: 1.1678 - val_accuracy: 0.5881
Epoch 7/10
57/57 [==============================] - 577s 10s/step - loss: 1.1320 - accuracy: 0.5595 - val_loss: 1.1756 - val_accuracy: 0.5378
Epoch 

In [20]:
from tensorflow.keras.applications import VGG16

vgg = VGG16(input_shape = (224,224,3), weights = "imagenet", include_top = False)
for layer in vgg.layers:
 layer.trainable = False

x = tf.keras.layers.Flatten()(vgg.output)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(13, activation = "softmax")(x)

model = Model(inputs = vgg.input, outputs = x)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [21]:

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True,verbose=1)

model.fit(train_data_gen, validation_data=validation_data_gen, epochs=10, verbose=1, callbacks=[early_stopping_cb])

Epoch 1/10
57/57 [==============================] - 588s 10s/step - loss: 2.5408 - accuracy: 0.1913 - val_loss: 2.8533 - val_accuracy: 0.2494
Epoch 2/10
57/57 [==============================] - 574s 10s/step - loss: 2.0378 - accuracy: 0.3065 - val_loss: 2.0281 - val_accuracy: 0.3524
Epoch 3/10
57/57 [==============================] - 574s 10s/step - loss: 1.7350 - accuracy: 0.3932 - val_loss: 1.7665 - val_accuracy: 0.4005
Epoch 4/10
57/57 [==============================] - 575s 10s/step - loss: 1.5095 - accuracy: 0.4700 - val_loss: 1.4811 - val_accuracy: 0.4416
Epoch 5/10
57/57 [==============================] - 573s 10s/step - loss: 1.3235 - accuracy: 0.5228 - val_loss: 1.3536 - val_accuracy: 0.4966
Epoch 6/10
57/57 [==============================] - 575s 10s/step - loss: 1.2108 - accuracy: 0.5645 - val_loss: 1.2290 - val_accuracy: 0.5629
Epoch 7/10
57/57 [==============================] - 572s 10s/step - loss: 1.1065 - accuracy: 0.6057 - val_loss: 1.1114 - val_accuracy: 0.5835
Epoch 

In [31]:
from tensorflow.keras.applications import VGG16

vgg = VGG16(input_shape = (224,224,3), weights = "imagenet", include_top = False)
for layer in vgg.layers:
 layer.trainable = False

x = tf.keras.layers.Flatten()(vgg.output)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.Dense(13, activation = "softmax")(x)

model = Model(inputs = vgg.input, outputs = x)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.95, nesterov=True)

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

In [32]:
training_steps_per_epoch = np.ceil(train_data_gen.samples / 32)
validation_steps_per_epoch = np.ceil(validation_data_gen.samples / 32)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True,verbose=1)

model.fit(train_data_gen, steps_per_epoch = training_steps_per_epoch, validation_data=validation_data_gen, validation_steps=validation_steps_per_epoch, epochs=15, verbose=1, callbacks=[early_stopping_cb])

Epoch 1/15
57/57 [==============================] - 579s 10s/step - loss: 2.9833 - accuracy: 0.1752 - val_loss: 2.8231 - val_accuracy: 0.2998
Epoch 2/15
57/57 [==============================] - 575s 10s/step - loss: 2.0946 - accuracy: 0.2992 - val_loss: 1.9260 - val_accuracy: 0.3684
Epoch 3/15
57/57 [==============================] - 573s 10s/step - loss: 1.6875 - accuracy: 0.4644 - val_loss: 1.7206 - val_accuracy: 0.4508
Epoch 4/15
57/57 [==============================] - 573s 10s/step - loss: 1.4399 - accuracy: 0.5189 - val_loss: 1.6061 - val_accuracy: 0.4828
Epoch 5/15
57/57 [==============================] - 573s 10s/step - loss: 1.2113 - accuracy: 0.5996 - val_loss: 1.3917 - val_accuracy: 0.5606
Epoch 6/15
57/57 [==============================] - 573s 10s/step - loss: 1.0378 - accuracy: 0.6635 - val_loss: 1.2431 - val_accuracy: 0.6041
Epoch 7/15
57/57 [==============================] - 570s 10s/step - loss: 0.8694 - accuracy: 0.7097 - val_loss: 1.0956 - val_accuracy: 0.6430
Epoch 

In [33]:
trdata = ImageDataGenerator(validation_split=0.2)
train_data_gen = trdata.flow_from_directory(directory="khateeb_data", target_size=(224,224), shuffle=True, class_mode="categorical", subset="training")
validation_data_gen = trdata.flow_from_directory(directory="khateeb_data", target_size=(224,224), shuffle=True, class_mode="categorical", subset="validation")

Found 1798 images belonging to 13 classes.
Found 437 images belonging to 13 classes.


In [34]:
from tensorflow.keras.applications import VGG16

vgg = VGG16(input_shape = (224,224,3), weights = "imagenet", include_top = False)
for layer in vgg.layers:
 layer.trainable = False

x = tf.keras.layers.Flatten()(vgg.output)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.Dense(13, activation = "softmax")(x)

model = Model(inputs = vgg.input, outputs = x)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.95, nesterov=True)

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

In [35]:
training_steps_per_epoch = np.ceil(train_data_gen.samples / 32)
validation_steps_per_epoch = np.ceil(validation_data_gen.samples / 32)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True,verbose=1)

model.fit(train_data_gen, steps_per_epoch = training_steps_per_epoch, validation_data=validation_data_gen, validation_steps=validation_steps_per_epoch, epochs=15, verbose=1, callbacks=[early_stopping_cb])

Epoch 1/15
57/57 [==============================] - 561s 10s/step - loss: 2.4866 - accuracy: 0.2408 - val_loss: 2.7462 - val_accuracy: 0.3227
Epoch 2/15
57/57 [==============================] - 550s 10s/step - loss: 1.5551 - accuracy: 0.4950 - val_loss: 1.6578 - val_accuracy: 0.4645
Epoch 3/15
57/57 [==============================] - 550s 10s/step - loss: 1.0954 - accuracy: 0.6468 - val_loss: 1.4490 - val_accuracy: 0.5286
Epoch 4/15
57/57 [==============================] - 550s 10s/step - loss: 0.8990 - accuracy: 0.7303 - val_loss: 1.3116 - val_accuracy: 0.5744
Epoch 5/15
57/57 [==============================] - 556s 10s/step - loss: 0.6971 - accuracy: 0.7875 - val_loss: 1.2583 - val_accuracy: 0.5904
Epoch 6/15
57/57 [==============================] - 554s 10s/step - loss: 0.5235 - accuracy: 0.8515 - val_loss: 1.2437 - val_accuracy: 0.6087
Epoch 7/15
57/57 [==============================] - 554s 10s/step - loss: 0.4477 - accuracy: 0.8704 - val_loss: 1.2875 - val_accuracy: 0.5927
Epoch 

KeyboardInterrupt: 

In [37]:
trdata = ImageDataGenerator(rotation_range = 30, validation_split=0.25)
train_data_gen = trdata.flow_from_directory(directory="khateeb_data", target_size=(224,224), shuffle=True, class_mode="categorical", subset="training")
validation_data_gen = trdata.flow_from_directory(directory="khateeb_data", target_size=(224,224), shuffle=True, class_mode="categorical", subset="validation")

Found 1680 images belonging to 13 classes.
Found 555 images belonging to 13 classes.


In [38]:
from tensorflow.keras.applications import VGG16

vgg = VGG16(input_shape = (224,224,3), weights = "imagenet", include_top = False)
for layer in vgg.layers:
 layer.trainable = False

x = tf.keras.layers.Flatten()(vgg.output)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.Dense(13, activation = "softmax")(x)

model = Model(inputs = vgg.input, outputs = x)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.95, nesterov=True)

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

In [39]:
training_steps_per_epoch = np.ceil(train_data_gen.samples / 32)
validation_steps_per_epoch = np.ceil(validation_data_gen.samples / 32)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True,verbose=1)

model.fit(train_data_gen, steps_per_epoch = training_steps_per_epoch, validation_data=validation_data_gen, validation_steps=validation_steps_per_epoch, epochs=15, verbose=1, callbacks=[early_stopping_cb])

Epoch 1/15
53/53 [==============================] - 579s 11s/step - loss: 2.8069 - accuracy: 0.2030 - val_loss: 3.3231 - val_accuracy: 0.2703
Epoch 2/15
53/53 [==============================] - 579s 11s/step - loss: 1.8083 - accuracy: 0.4161 - val_loss: 2.1332 - val_accuracy: 0.3405
Epoch 3/15
53/53 [==============================] - 575s 11s/step - loss: 1.3750 - accuracy: 0.5631 - val_loss: 1.8259 - val_accuracy: 0.3892
Epoch 4/15
53/53 [==============================] - 574s 11s/step - loss: 1.0913 - accuracy: 0.6565 - val_loss: 1.6786 - val_accuracy: 0.4432
Epoch 5/15
53/53 [==============================] - 576s 11s/step - loss: 0.8337 - accuracy: 0.7548 - val_loss: 1.5887 - val_accuracy: 0.5207
Epoch 6/15
 4/53 [=>............................] - ETA: 6:41 - loss: 0.6581 - accuracy: 0.7812

KeyboardInterrupt: 

In [40]:
trdata = ImageDataGenerator(rotation_range = 30, validation_split=0.25)
train_data_gen = trdata.flow_from_directory(directory="khateeb_data", target_size=(299,299), shuffle=True, class_mode="categorical", subset="training")
validation_data_gen = trdata.flow_from_directory(directory="khateeb_data", target_size=(299,299), shuffle=True, class_mode="categorical", subset="validation")

Found 1680 images belonging to 13 classes.
Found 555 images belonging to 13 classes.


In [43]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model

incep = InceptionV3(input_shape = (299,299,3), weights = "imagenet", include_top = False)
for layer in incep.layers:
 layer.trainable = False

x = tf.keras.layers.Flatten()(incep.output)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.Dense(13, activation = "softmax")(x)

incep_model = Model(inputs = incep.input, outputs = x)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

incep_model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [44]:
training_steps_per_epoch = np.ceil(train_data_gen.samples / 32)
validation_steps_per_epoch = np.ceil(validation_data_gen.samples / 32)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True,verbose=1)

incep_model.fit(train_data_gen, steps_per_epoch = training_steps_per_epoch, validation_data=validation_data_gen, validation_steps=validation_steps_per_epoch, epochs=15, verbose=1, callbacks=[early_stopping_cb])

Epoch 1/15
53/53 [==============================] - 516s 10s/step - loss: 2.6138 - accuracy: 0.1655 - val_loss: 3.5190 - val_accuracy: 0.1802
Epoch 2/15
53/53 [==============================] - 507s 10s/step - loss: 2.0143 - accuracy: 0.2964 - val_loss: 2.2216 - val_accuracy: 0.2054
Epoch 3/15
53/53 [==============================] - 506s 10s/step - loss: 1.7748 - accuracy: 0.3607 - val_loss: 2.1622 - val_accuracy: 0.1730
Epoch 4/15
53/53 [==============================] - 503s 10s/step - loss: 1.6433 - accuracy: 0.3976 - val_loss: 1.9780 - val_accuracy: 0.2144
Epoch 5/15
53/53 [==============================] - 508s 10s/step - loss: 1.4987 - accuracy: 0.4500 - val_loss: 2.1101 - val_accuracy: 0.1802
Epoch 6/15
53/53 [==============================] - 505s 10s/step - loss: 1.4371 - accuracy: 0.4833 - val_loss: 2.1652 - val_accuracy: 0.1892
Epoch 7/15
53/53 [==============================] - 505s 10s/step - loss: 1.3252 - accuracy: 0.5083 - val_loss: 2.1866 - val_accuracy: 0.2036
Epoch 

In [46]:
trdata = ImageDataGenerator(zoom_range = 0.2, validation_split=0.25)
train_data_gen = trdata.flow_from_directory(directory="khateeb_data", target_size=(244,244), shuffle=True, class_mode="categorical", subset="training")
validation_data_gen = trdata.flow_from_directory(directory="khateeb_data", target_size=(244,244), shuffle=True, class_mode="categorical", subset="validation")

Found 1680 images belonging to 13 classes.
Found 555 images belonging to 13 classes.


In [47]:
from tensorflow.keras.applications import VGG16

vgg = VGG16(input_shape = (224,224,3), weights = "imagenet", include_top = False)
for layer in vgg.layers:
 layer.trainable = False

x = tf.keras.layers.Flatten()(vgg.output)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.Dense(13, activation = "softmax")(x)

model = Model(inputs = vgg.input, outputs = x)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.95, nesterov=True)

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

In [49]:
training_steps_per_epoch = np.ceil(train_data_gen.samples / 32)
validation_steps_per_epoch = np.ceil(validation_data_gen.samples / 32)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True,verbose=1)

model.fit(train_data_gen, steps_per_epoch = training_steps_per_epoch, validation_data=validation_data_gen, validation_steps=validation_steps_per_epoch, epochs=15, verbose=1, callbacks=[early_stopping_cb])

Epoch 1/15
53/53 [==============================] - 643s 12s/step - loss: 2.5894 - accuracy: 0.2298 - val_loss: 2.5765 - val_accuracy: 0.2685
Epoch 2/15
53/53 [==============================] - 638s 12s/step - loss: 1.6580 - accuracy: 0.4768 - val_loss: 1.7993 - val_accuracy: 0.4216
Epoch 3/15
53/53 [==============================] - 640s 12s/step - loss: 1.2158 - accuracy: 0.6173 - val_loss: 1.4886 - val_accuracy: 0.4937
Epoch 4/15
53/53 [==============================] - 637s 12s/step - loss: 0.9582 - accuracy: 0.7137 - val_loss: 1.4485 - val_accuracy: 0.4973
Epoch 5/15
53/53 [==============================] - 637s 12s/step - loss: 0.7296 - accuracy: 0.7798 - val_loss: 1.4142 - val_accuracy: 0.5279
Epoch 6/15
53/53 [==============================] - 639s 12s/step - loss: 0.5923 - accuracy: 0.8345 - val_loss: 1.3189 - val_accuracy: 0.5658
Epoch 7/15
53/53 [==============================] - 637s 12s/step - loss: 0.4947 - accuracy: 0.8500 - val_loss: 1.3703 - val_accuracy: 0.5766
Epoch 

In [50]:
from tensorflow.keras.applications import VGG16

vgg = VGG16(input_shape = (224,224,3), weights = "imagenet", include_top = False)
for layer in vgg.layers:
 layer.trainable = False

x = tf.keras.layers.Flatten()(vgg.output)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.7)(x)
x = tf.keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.7)(x)
x = tf.keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.7)(x)
x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.Dense(13, activation = "softmax")(x)

model = Model(inputs = vgg.input, outputs = x)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.95, nesterov=True)

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

In [51]:
training_steps_per_epoch = np.ceil(train_data_gen.samples / 32)
validation_steps_per_epoch = np.ceil(validation_data_gen.samples / 32)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True,verbose=1)

model.fit(train_data_gen, steps_per_epoch = training_steps_per_epoch, validation_data=validation_data_gen, validation_steps=validation_steps_per_epoch, epochs=15, verbose=1, callbacks=[early_stopping_cb])

Epoch 1/15
53/53 [==============================] - 642s 12s/step - loss: 3.4332 - accuracy: 0.1048 - val_loss: 3.6345 - val_accuracy: 0.1784
Epoch 2/15
53/53 [==============================] - 634s 12s/step - loss: 2.6597 - accuracy: 0.1863 - val_loss: 2.3103 - val_accuracy: 0.2775
Epoch 3/15
53/53 [==============================] - 633s 12s/step - loss: 2.3478 - accuracy: 0.2429 - val_loss: 2.1086 - val_accuracy: 0.3135
Epoch 4/15
53/53 [==============================] - 634s 12s/step - loss: 2.1321 - accuracy: 0.3071 - val_loss: 1.9780 - val_accuracy: 0.3514
Epoch 5/15
53/53 [==============================] - 631s 12s/step - loss: 1.9579 - accuracy: 0.3411 - val_loss: 1.8955 - val_accuracy: 0.3622
Epoch 6/15
53/53 [==============================] - 633s 12s/step - loss: 1.8372 - accuracy: 0.3786 - val_loss: 1.8189 - val_accuracy: 0.3784
Epoch 7/15
53/53 [==============================] - 630s 12s/step - loss: 1.6840 - accuracy: 0.4268 - val_loss: 1.7632 - val_accuracy: 0.4306
Epoch 

In [52]:
from tensorflow.keras.applications import VGG16

vgg = VGG16(input_shape = (224,224,3), weights = "imagenet", include_top = False)
for layer in vgg.layers:
 layer.trainable = False

x = tf.keras.layers.Flatten()(vgg.output)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.7)(x)
x = tf.keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.7)(x)
x = tf.keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.7)(x)
x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.7)(x)
x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.7)(x)
x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.Dense(13, activation = "softmax")(x)

model = Model(inputs = vgg.input, outputs = x)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.95, nesterov=True)

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

In [53]:
training_steps_per_epoch = np.ceil(train_data_gen.samples / 32)
validation_steps_per_epoch = np.ceil(validation_data_gen.samples / 32)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True,verbose=1)

model.fit(train_data_gen, steps_per_epoch = training_steps_per_epoch, validation_data=validation_data_gen, validation_steps=validation_steps_per_epoch, epochs=15, verbose=1, callbacks=[early_stopping_cb])

Epoch 1/15
53/53 [==============================] - 649s 12s/step - loss: 3.7259 - accuracy: 0.0827 - val_loss: 2.7338 - val_accuracy: 0.0991
Epoch 2/15
53/53 [==============================] - 633s 12s/step - loss: 3.2675 - accuracy: 0.0798 - val_loss: 2.5755 - val_accuracy: 0.1063
Epoch 3/15
53/53 [==============================] - 631s 12s/step - loss: 3.0851 - accuracy: 0.0810 - val_loss: 2.5906 - val_accuracy: 0.0703
Epoch 4/15
53/53 [==============================] - 628s 12s/step - loss: 3.0271 - accuracy: 0.0738 - val_loss: 2.5940 - val_accuracy: 0.0505
Epoch 5/15
53/53 [==============================] - 627s 12s/step - loss: 2.8725 - accuracy: 0.0851 - val_loss: 2.5790 - val_accuracy: 0.0667
Epoch 6/15
53/53 [==============================] - 625s 12s/step - loss: 2.8289 - accuracy: 0.0714 - val_loss: 2.5742 - val_accuracy: 0.1171
Epoch 7/15
53/53 [==============================] - 625s 12s/step - loss: 2.7553 - accuracy: 0.0798 - val_loss: 2.5693 - val_accuracy: 0.0631
Epoch 

KeyboardInterrupt: 

In [56]:
from tensorflow.keras.applications import VGG16

vgg = VGG16(input_shape = (224,224,3), weights = "imagenet", include_top = False)
for layer in vgg.layers:
 layer.trainable = False

x = tf.keras.layers.Flatten()(vgg.output)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.Dense(13, activation = "softmax")(x)

model = Model(inputs = vgg.input, outputs = x)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.95, nesterov=True)

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

In [57]:
training_steps_per_epoch = np.ceil(train_data_gen.samples / 32)
validation_steps_per_epoch = np.ceil(validation_data_gen.samples / 32)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True,verbose=1)

model.fit(train_data_gen, steps_per_epoch = training_steps_per_epoch, validation_data=validation_data_gen, validation_steps=validation_steps_per_epoch, epochs=15, verbose=1, callbacks=[early_stopping_cb])

Epoch 1/15
53/53 [==============================] - 634s 12s/step - loss: 2.1397 - accuracy: 0.3280 - val_loss: 3.9212 - val_accuracy: 0.3225
Epoch 2/15
53/53 [==============================] - 624s 12s/step - loss: 1.2078 - accuracy: 0.6530 - val_loss: 2.3165 - val_accuracy: 0.4000
Epoch 3/15
53/53 [==============================] - 624s 12s/step - loss: 0.7689 - accuracy: 0.7982 - val_loss: 1.7876 - val_accuracy: 0.4505
Epoch 4/15
53/53 [==============================] - 623s 12s/step - loss: 0.5303 - accuracy: 0.8702 - val_loss: 1.7462 - val_accuracy: 0.4865
Epoch 5/15
53/53 [==============================] - 622s 12s/step - loss: 0.3673 - accuracy: 0.9060 - val_loss: 1.6435 - val_accuracy: 0.4919
Epoch 6/15
53/53 [==============================] - 622s 12s/step - loss: 0.2807 - accuracy: 0.9286 - val_loss: 1.5580 - val_accuracy: 0.5225
Epoch 7/15
53/53 [==============================] - 623s 12s/step - loss: 0.2131 - accuracy: 0.9488 - val_loss: 1.7168 - val_accuracy: 0.5207
Epoch 

In [58]:
from tensorflow.keras.applications import VGG16

vgg = VGG16(input_shape = (224,224,3), weights = "imagenet", include_top = False)
for layer in vgg.layers:
 layer.trainable = False

x = tf.keras.layers.Flatten()(vgg.output)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(13, activation = "softmax")(x)

model = Model(inputs = vgg.input, outputs = x)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.95, nesterov=True)

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

In [59]:
training_steps_per_epoch = np.ceil(train_data_gen.samples / 32)
validation_steps_per_epoch = np.ceil(validation_data_gen.samples / 32)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True,verbose=1)

model.fit(train_data_gen, steps_per_epoch = training_steps_per_epoch, validation_data=validation_data_gen, validation_steps=validation_steps_per_epoch, epochs=15, verbose=1, callbacks=[early_stopping_cb])

Epoch 1/15
53/53 [==============================] - 636s 12s/step - loss: 2.0370 - accuracy: 0.3589 - val_loss: 3.1515 - val_accuracy: 0.3063
Epoch 2/15
53/53 [==============================] - 625s 12s/step - loss: 1.2292 - accuracy: 0.6673 - val_loss: 2.2169 - val_accuracy: 0.3802
Epoch 3/15
53/53 [==============================] - 627s 12s/step - loss: 0.8834 - accuracy: 0.7738 - val_loss: 1.8460 - val_accuracy: 0.4613
Epoch 4/15
53/53 [==============================] - 628s 12s/step - loss: 0.6346 - accuracy: 0.8524 - val_loss: 1.7339 - val_accuracy: 0.4883
Epoch 5/15
53/53 [==============================] - 625s 12s/step - loss: 0.4394 - accuracy: 0.9125 - val_loss: 1.7410 - val_accuracy: 0.5225
Epoch 6/15
53/53 [==============================] - 627s 12s/step - loss: 0.3575 - accuracy: 0.9339 - val_loss: 1.7289 - val_accuracy: 0.5225
Epoch 7/15
26/53 [=============>................] - ETA: 4:07 - loss: 0.2796 - accuracy: 0.9495

KeyboardInterrupt: 

In [60]:
trdata = ImageDataGenerator(zoom_range = 0.3, validation_split=0.3)
train_data_gen = trdata.flow_from_directory(directory="khateeb_data", target_size=(244,244), shuffle=True, class_mode="categorical", subset="training")
validation_data_gen = trdata.flow_from_directory(directory="khateeb_data", target_size=(244,244), shuffle=True, class_mode="categorical", subset="validation")

Found 1573 images belonging to 13 classes.
Found 662 images belonging to 13 classes.


In [61]:
from tensorflow.keras.applications import VGG16

vgg = VGG16(input_shape = (224,224,3), weights = "imagenet", include_top = False)
for layer in vgg.layers:
 layer.trainable = False

x = tf.keras.layers.Flatten()(vgg.output)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(16, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(16, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(16, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(13, activation = "softmax")(x)

model = Model(inputs = vgg.input, outputs = x)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.95, nesterov=True)

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

In [62]:
training_steps_per_epoch = np.ceil(train_data_gen.samples / 32)
validation_steps_per_epoch = np.ceil(validation_data_gen.samples / 32)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True,verbose=1)

model.fit(train_data_gen, steps_per_epoch = training_steps_per_epoch, validation_data=validation_data_gen, validation_steps=validation_steps_per_epoch, epochs=15, verbose=1, callbacks=[early_stopping_cb])

Epoch 1/15
50/50 [==============================] - 638s 13s/step - loss: 2.4459 - accuracy: 0.1964 - val_loss: 3.6285 - val_accuracy: 0.1541
Epoch 2/15
50/50 [==============================] - 631s 13s/step - loss: 2.0912 - accuracy: 0.3109 - val_loss: 2.2343 - val_accuracy: 0.2689
Epoch 3/15
50/50 [==============================] - 628s 13s/step - loss: 1.9095 - accuracy: 0.3566 - val_loss: 2.1856 - val_accuracy: 0.2946
Epoch 4/15
50/50 [==============================] - 627s 13s/step - loss: 1.8071 - accuracy: 0.4170 - val_loss: 2.0528 - val_accuracy: 0.3112
Epoch 5/15
50/50 [==============================] - 626s 13s/step - loss: 1.6951 - accuracy: 0.4495 - val_loss: 1.9446 - val_accuracy: 0.3429
Epoch 6/15
50/50 [==============================] - 626s 13s/step - loss: 1.5845 - accuracy: 0.4838 - val_loss: 1.9475 - val_accuracy: 0.3595
Epoch 7/15
50/50 [==============================] - 624s 13s/step - loss: 1.4589 - accuracy: 0.5283 - val_loss: 1.8889 - val_accuracy: 0.3505
Epoch 

In [63]:
from tensorflow.keras.applications import VGG16

vgg = VGG16(input_shape = (224,224,3), weights = "imagenet", include_top = False)
for layer in vgg.layers:
 layer.trainable = False

x = tf.keras.layers.Flatten()(vgg.output)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(16, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(13, activation = "softmax")(x)

model = Model(inputs = vgg.input, outputs = x)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.95, nesterov=True)

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

In [64]:
training_steps_per_epoch = np.ceil(train_data_gen.samples / 32)
validation_steps_per_epoch = np.ceil(validation_data_gen.samples / 32)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True,verbose=1)

model.fit(train_data_gen, steps_per_epoch = training_steps_per_epoch, validation_data=validation_data_gen, validation_steps=validation_steps_per_epoch, epochs=15, verbose=1, callbacks=[early_stopping_cb])

Epoch 1/15
50/50 [==============================] - 637s 13s/step - loss: 2.3900 - accuracy: 0.2219 - val_loss: 3.7924 - val_accuracy: 0.2508
Epoch 2/15
50/50 [==============================] - 625s 13s/step - loss: 1.8711 - accuracy: 0.4107 - val_loss: 2.0142 - val_accuracy: 0.3701
Epoch 3/15
50/50 [==============================] - 624s 13s/step - loss: 1.6161 - accuracy: 0.5175 - val_loss: 1.8161 - val_accuracy: 0.4305
Epoch 4/15
50/50 [==============================] - 627s 13s/step - loss: 1.4021 - accuracy: 0.6122 - val_loss: 1.6588 - val_accuracy: 0.4592
Epoch 5/15
50/50 [==============================] - 625s 13s/step - loss: 1.2538 - accuracy: 0.6561 - val_loss: 1.6112 - val_accuracy: 0.4773
Epoch 6/15
50/50 [==============================] - 622s 12s/step - loss: 1.0486 - accuracy: 0.7292 - val_loss: 1.5571 - val_accuracy: 0.4653
Epoch 7/15
50/50 [==============================] - 626s 13s/step - loss: 0.9176 - accuracy: 0.7641 - val_loss: 1.4832 - val_accuracy: 0.5030
Epoch 

In [65]:
from tensorflow.keras.applications import VGG16

vgg = VGG16(input_shape = (224,224,3), weights = "imagenet", include_top = False)
for layer in vgg.layers:
 layer.trainable = False

x = tf.keras.layers.Flatten()(vgg.output)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(50, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(50, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(50, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(50, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(13, activation = "softmax")(x)

model = Model(inputs = vgg.input, outputs = x)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.95, nesterov=True)

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

In [66]:
training_steps_per_epoch = np.ceil(train_data_gen.samples / 32)
validation_steps_per_epoch = np.ceil(validation_data_gen.samples / 32)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True,verbose=1)

model.fit(train_data_gen, steps_per_epoch = training_steps_per_epoch, validation_data=validation_data_gen, validation_steps=validation_steps_per_epoch, epochs=15, verbose=1, callbacks=[early_stopping_cb])

Epoch 1/15
50/50 [==============================] - 647s 13s/step - loss: 2.6834 - accuracy: 0.1608 - val_loss: 3.6446 - val_accuracy: 0.2039
Epoch 2/15
50/50 [==============================] - 634s 13s/step - loss: 2.1684 - accuracy: 0.2943 - val_loss: 2.2294 - val_accuracy: 0.2795
Epoch 3/15
50/50 [==============================] - 634s 13s/step - loss: 1.8226 - accuracy: 0.4069 - val_loss: 2.0906 - val_accuracy: 0.3082
Epoch 4/15
50/50 [==============================] - 637s 13s/step - loss: 1.6611 - accuracy: 0.4577 - val_loss: 2.0813 - val_accuracy: 0.3127
Epoch 5/15
50/50 [==============================] - 632s 13s/step - loss: 1.4080 - accuracy: 0.5493 - val_loss: 1.9909 - val_accuracy: 0.3550
Epoch 6/15
50/50 [==============================] - 634s 13s/step - loss: 1.2708 - accuracy: 0.6046 - val_loss: 1.9354 - val_accuracy: 0.3444
Epoch 7/15
50/50 [==============================] - 636s 13s/step - loss: 1.0926 - accuracy: 0.6529 - val_loss: 1.8030 - val_accuracy: 0.3973
Epoch 

In [67]:
import os

current_directory = os.getcwd()
path = os.path.join(current_directory, "data")
print(path)

C:\Users\Moath\Downloads\data


In [69]:
folders = ['train', 'test']
image_exts = ['jpeg','jpg', 'bmp', 'png']

for image_class in os.listdir(path): 
  print(image_class)
  for image in os.listdir(os.path.join(path, image_class)):
      image_path = os.path.join(path, image_class, image)
      try: 
          img = cv2.imread(image_path)
          tip = imghdr.what(image_path)
          if tip not in image_exts: 
              print('Image not in ext list {}'.format(image_path))
              os.remove(image_path)
      except Exception as e: 
          print('Issue with image {}'.format(image_path))
          # os.remove(image_path)

black_bishop
black_king
black_knight
Image not in ext list C:\Users\Moath\Downloads\data\black_knight\.DS_Store
black_pawn
black_queen
black_rook
empty
white_bishop
white_king
white_knight
white_pawn
white_queen
white_rook


In [72]:
trdata = ImageDataGenerator(zoom_range = 0.3, validation_split=0.3)
train_data_gen = trdata.flow_from_directory(directory="data", target_size=(244,244), shuffle=True, class_mode="categorical", subset="training")
validation_data_gen = trdata.flow_from_directory(directory="data", target_size=(244,244), shuffle=True, class_mode="categorical", subset="validation")

Found 2227 images belonging to 13 classes.
Found 940 images belonging to 13 classes.


In [73]:
from tensorflow.keras.applications import VGG16

vgg = VGG16(input_shape = (224,224,3), weights = "imagenet", include_top = False)
for layer in vgg.layers:
 layer.trainable = False

x = tf.keras.layers.Flatten()(vgg.output)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(50, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(50, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(50, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(50, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(13, activation = "softmax")(x)

model = Model(inputs = vgg.input, outputs = x)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.95, nesterov=True)

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

In [74]:
training_steps_per_epoch = np.ceil(train_data_gen.samples / 32)
validation_steps_per_epoch = np.ceil(validation_data_gen.samples / 32)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True,verbose=1)

model.fit(train_data_gen, steps_per_epoch = training_steps_per_epoch, validation_data=validation_data_gen, validation_steps=validation_steps_per_epoch, epochs=15, verbose=1, callbacks=[early_stopping_cb])

Epoch 1/15
70/70 [==============================] - 654s 9s/step - loss: 2.7477 - accuracy: 0.1320 - val_loss: 2.8544 - val_accuracy: 0.1777
Epoch 2/15
70/70 [==============================] - 612s 9s/step - loss: 2.2414 - accuracy: 0.2434 - val_loss: 1.9667 - val_accuracy: 0.3489
Epoch 3/15
70/70 [==============================] - 609s 9s/step - loss: 1.9318 - accuracy: 0.3278 - val_loss: 1.6875 - val_accuracy: 0.4468
Epoch 4/15
70/70 [==============================] - 612s 9s/step - loss: 1.6723 - accuracy: 0.4185 - val_loss: 1.5065 - val_accuracy: 0.5032
Epoch 5/15
70/70 [==============================] - 611s 9s/step - loss: 1.4946 - accuracy: 0.4859 - val_loss: 1.3235 - val_accuracy: 0.5606
Epoch 6/15
70/70 [==============================] - 615s 9s/step - loss: 1.2946 - accuracy: 0.5424 - val_loss: 1.2437 - val_accuracy: 0.5596
Epoch 7/15
70/70 [==============================] - 619s 9s/step - loss: 1.1378 - accuracy: 0.6075 - val_loss: 1.1647 - val_accuracy: 0.5872
Epoch 8/15
70

In [77]:
from tensorflow.keras.applications import VGG16

vgg = VGG16(input_shape = (224,224,3), weights = "imagenet", include_top = False)
for layer in vgg.layers:
 layer.trainable = False

x = tf.keras.layers.Flatten()(vgg.output)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(16, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(13, activation = "softmax")(x)

model = Model(inputs = vgg.input, outputs = x)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.02, momentum=0.90, nesterov=True)

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

In [ ]:
training_steps_per_epoch = np.ceil(train_data_gen.samples / 32)
validation_steps_per_epoch = np.ceil(validation_data_gen.samples / 32)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True,verbose=1)

model.fit(train_data_gen, steps_per_epoch = training_steps_per_epoch, validation_data=validation_data_gen, validation_steps=validation_steps_per_epoch, epochs=15, verbose=1, callbacks=[early_stopping_cb])

Epoch 1/15
70/70 [==============================] - 607s 9s/step - loss: 2.7291 - accuracy: 0.1132 - val_loss: 2.5023 - val_accuracy: 0.1723
Epoch 2/15
70/70 [==============================] - 605s 9s/step - loss: 2.4086 - accuracy: 0.1608 - val_loss: 2.1897 - val_accuracy: 0.2702
Epoch 3/15
70/70 [==============================] - 601s 9s/step - loss: 2.2193 - accuracy: 0.2380 - val_loss: 1.9891 - val_accuracy: 0.3606
Epoch 4/15
70/70 [==============================] - 605s 9s/step - loss: 2.0259 - accuracy: 0.2955 - val_loss: 1.8615 - val_accuracy: 0.3851
Epoch 5/15
31/70 [============>.................] - ETA: 3:57 - loss: 1.9495 - accuracy: 0.3136